In [47]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [48]:
data = pd.read_csv('input/train.csv')
# data = pd.read_csv('input/train_min.csv')  # small data

X, y = data.iloc[:,2:].values, data.iloc[:,1].values

In [57]:
# cv: 0.8333937317304528
# logistic = SGDClassifier(loss='log', max_iter=50000, tol=1e-3, alpha=0.1)

#cv: 0.8492245702937613
# logistic = SGDClassifier(loss='log', max_iter=50000, tol=1e-3, alpha=0.01)

#cv: 0.8456784557444358
# logistic = SGDClassifier(loss='modified_huber', max_iter=5000, tol=1e-3, alpha=0.01)

#cv: 0.8494248938378591
# logistic = SGDClassifier(loss='log', max_iter=5000, tol=1e-3, alpha=0.01)

#cv: 0.8515980352095942
logistic = SGDClassifier(loss='log', max_iter=5000, tol=1e-7, alpha=0.01)


# temp
# scaler = preprocessing.StandardScaler().fit(X)


# try grid-search
params = { 'tol': [ 1e-8, 1e-9, 1e-10, 1e-15 ] }
cls = GridSearchCV(estimator=logistic, param_grid=params, cv=5, scoring='roc_auc')
cls.fit(X, y)

print(cls.best_params_)
print(cls.best_score_)

{'tol': 1e-15}
0.8514939360542757


In [ ]:
# create final output
test = pd.read_csv('input/test.csv')
test_ids, test_x = test.iloc[:,0], test.iloc[:,1:]

test_y = cls.predict_proba(test_x)

In [ ]:
# output the result
sub_ids = pd.DataFrame(test_ids)
sub_y = pd.DataFrame(pd.DataFrame(test_y).iloc[:,1])

sub = pd.DataFrame(np.hstack((sub_ids, sub_y)))
sub.to_csv('output.csv', header=['ID_code', 'target'], index=False)

output_check = pd.read_csv('output.csv')
output_check.head()